<a href="https://colab.research.google.com/github/KoMurase/Learn_NLP/blob/master/doc2vec_%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.5)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.3)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main

In [0]:
import zipfile 
import os.path 
import urllib.request as req 
import MeCab 
from gensim import models 
from gensim.models.doc2vec import TaggedDocument 



In [0]:
#MeCabの初期化
 
mecab = MeCab.Tagger()   
mecab.parse("")

list = [{
    "author": {
        "name":"宮沢賢治",
        "url": "https://www.aozora.gr.jp/cards/000081/files/",
    },
    "books":[
             {"name":"銀河鉄道の夜","zipname":"43737_ruby_19028.zip"},
             {"name":"注文の多い料理店","zipname":"1927_ruby_17835.zip"}
             ]
},
{
    "author": {
        "name":"芥川龍之介",
        "url": "https://www.aozora.gr.jp/cards/000879/files/",
    },
    "books":[
             {"name":"羅生門","zipname":"127_ruby_150.zip"},
             {"name":"鼻","zipname":"42_ruby_154.zip"}
             ]
},

]

In [0]:
#取得したファイルループ処理に渡す
def book_list():
  for novelist in list:
    author = novelist["author"]
    for book in novelist["books"]:
      yield author, book   #大きな値を返してしまうとメモリの消費がぱないのでyieldを使う

In [0]:
#zipファイルを開いて中の文書を取得する
def read_book(auther, book):
  zipname = book["zipname"]
  #zipファイルがなければ取得する
  if not os.path.exists(zipname):
    req.urlretrieve(auther["url"] + zipname, zipname)
  zipname = book["zipname"]
  #zipファイルを開く
  with zipfile.ZipFile(zipname,"r") as zf:
    for filename in zf.namelist():
      with zf.open(filename, "r") as f:
        return f.read().decode("shift-jis")

In [0]:
# 分かち書きをする
def split_words(text):
  node = mecab.parseToNode(text)
  wakati_words = []
  while node is not None:
    hinshi = node.feature.split(",")[0]
    if hinshi in ["名詞"]:
      wakati_words.append(node.surface)
    elif hinshi in ["形容詞", "動詞"]:
      wakati_words.append(node.feature.split(",")[6])
    node = node.next 
  return wakati_words

In [27]:
#作品リストDoc2vecが読めるTaggedDocument形式にし,配列に追加する
documents = [] 
for auther, book in book_list():
  words = read_book(auther,book)
  wakati_words = split_words(words)
  document = TaggedDocument(
      wakati_words,[auther["name"] + ":" + book["name"]]
  )
  documents.append(document)

model = models.Doc2Vec(
    documents,dm=1, vector_size=300,window=5,min_count=1
)
#dmはDoc2Vecで扱うアルゴリズム 1=dmpw, 0=DBOW
#sizeはベクトルの次元を設定します Doc2Vecでは基本的に300が良いとされている
#windowは学習する単語の前後数, dmpwでは5が良いとされている
#min_countは最低何回出てきた文字列を対象とするかを設定する

model.save("/content/drive/My Drive/akutagawa/aozora.model")

print("完了")

完了


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [30]:
type(documents)


list

In [31]:
type(wakati_words)

list

In [32]:
len(wakati_words)

2123

In [39]:
print(wakati_words[0],wakati_words[1],wakati_words[50])

鼻 芥川 列


作者を分類してみる

In [47]:
model = models.Doc2Vec.load("/content/drive/My Drive/akutagawa/aozora.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def read_book2(url, zipname):
  if not os.path.exists(zipname):
    req.urlretrieve(url,zipname)
  with zipfile.ZipFile(zipname,"r") as zf:
    for filename in zf.namelist():
      with zf.open(filename, "r") as f:
        return f.read().decode("shift-jis")

In [0]:
def similar(title, url):
  zipname = url.split("/")[-1]

  words = read_book2(url,zipname)
  wakati_words = split_words(words)
  vector = split_words(words)
  vector = model.infer_vector(wakati_words)

  print(title+"と似た作品は...?")
  print(model.docvecs.most_similar([vector],topn=3))

In [52]:
similar("芥川龍之介:犬と猫",
        "https://www.aozora.gr.jp/cards/000879/files/56_ruby_845.zip")

芥川龍之介:犬と猫と似た作品は...?
[('芥川龍之介:鼻', 0.9997143149375916), ('芥川龍之介:羅生門', 0.9996194839477539), ('宮沢賢治:注文の多い料理店', 0.9991310238838196)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
